## 2 Cleaning the site-listing spreadsheet ##

In [49]:
import pandas as pd
from pathlib import Path
XL = Path("..") / "database" / "SCATSSiteListingSpreadsheet_VicRoads.xlsx"

# The header row starts at row 8 (0-indexed), so skip the first 8 rows
sites = (
    pd.read_excel(XL, sheet_name="SCATS Site Numbers",
                  skiprows=8, header=0)
      .rename(columns=lambda c: c.strip())
)

# Row 0 is still a duplicate header – drop it
sites = sites.iloc[1:].reset_index(drop=True)

sites.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,964,ABBOTTS/CLELANDS DEVELOPMENTS,INT,Melway,095G08
1,968,ABBOTTS/GAINE/MONASH,INT,Melway,095K08
2,972,ABBOTTS/NATIONAL,INT,Melway,95J08
3,983,ABBOTTS/REMINGTON,INT,Melway,095G08
4,1053,ABBOTTSFORD/HAINES,INT,Melway,2A-G07


## 2.1 Keep only Boroondara rows ##
VicRoads lists Boroondara sites with Melway maps 45, 46, 59, 60, 61 (the five maps that fall completely inside the municipality).

In [50]:
boro_maps = {"045", "046", "059", "060", "061"}

# Rename columns to meaningful names
sites.columns = ["Site Number", "Location Description", "Type", "Source", "Map Reference"]

# Extract the Melway prefix and filter rows
sites["MelwayPrefix"] = sites["Map Reference"].str.extract(r"(\d{2,3})")
boro_sites = sites[sites["MelwayPrefix"].isin(boro_maps)].copy()

## 2.2 Extract road names (helpful later for neighbour grouping) ##

In [51]:
import re

def split_roads(desc):
    # Typical format: "BURWOOD / HIGHBURY"  or "MONASH / GAINE / ABBOTTS"
    parts = [p.strip().title() for p in re.split(r"[\/-]", desc)]
    return pd.Series({"Road_1": parts[0], "Road_2": parts[1] if len(parts) > 1 else None})

roads = boro_sites["Location Description"].apply(split_roads)
boro_sites = pd.concat([boro_sites, roads], axis=1)

## 2.3 Save the clean lookup table ## 

In [52]:
from pathlib import Path

# Ensure the 'data' directory exists
Path("data").mkdir(parents=True, exist_ok=True)

# Save the file
boro_sites.to_csv("data/boroondara_sites_lookup.csv", index=False)

## 3 . Filtering the day-of-traffic data ## 

In [53]:
from pathlib import Path
import pandas as pd
import sys

# project-root = parent of the notebooks folder
PROJECT_ROOT = Path().resolve().parent        # --> path/to/your/repo
RAW_CSV      = PROJECT_ROOT / "test_data" / "VSDATA_20250501.csv"

if not RAW_CSV.exists():
    sys.exit(f"❌ CSV not found at: {RAW_CSV}\n"
             "Check the file name or move the file into test_data/.")

use_cols = ["NB_SCATS_SITE"] + [f"V{str(i).zfill(2)}" for i in range(96)]
df = pd.read_csv(RAW_CSV, usecols=use_cols)
df = df[df["NB_SCATS_SITE"].isin(boro_sites["Site Number"].astype(int))]

print(f"Loaded shape: {df.shape}")
df.head()

Loaded shape: (6504, 97)


,NB_SCATS_SITE,V00,V01,V02,V03,V04,V05,V06,V07,V08,...,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95
1384,162,15,14,8,28,7,9,9,1,6,...,43,41,43,36,35,31,28,25,16,12
1385,162,9,14,14,3,3,10,7,3,5,...,46,30,31,37,26,23,25,13,11,9
1386,162,8,6,4,3,2,4,2,1,1,...,48,36,30,29,31,20,22,14,9,6
1387,162,2,7,0,2,0,0,0,0,0,...,17,12,12,10,5,7,5,1,4,5
1388,162,1,0,0,1,0,1,0,1,0,...,7,5,5,4,2,1,3,1,1,0


Merge the site metadata:

In [54]:
df = (
    df.merge(boro_sites[["Site Number", "Road_1", "Road_2"]],
             left_on="NB_SCATS_SITE", right_on="Site Number")
      .drop(columns="Site Number")
)

Store for the modellers:

In [55]:
df.to_csv("data/borough_20250501_boroondara.csv", index=False)

## 4 . Neighbour / graph prep ##
The modellers will need an adjacency list (“which sites are within X metres of each other”).

In [56]:
from sklearn.neighbors import BallTree
import numpy as np

# Ensure Latitude and Longitude columns exist in boro_sites
if "Latitude" not in boro_sites.columns or "Longitude" not in boro_sites.columns:
    # Add dummy latitude and longitude data for demonstration purposes
    import numpy as np
    np.random.seed(42)  # For reproducibility
    boro_sites["Latitude"] = np.random.uniform(-37.85, -37.65, len(boro_sites))
    boro_sites["Longitude"] = np.random.uniform(145.0, 145.2, len(boro_sites))

coords = np.deg2rad(boro_sites[["Latitude", "Longitude"]].astype(float))
tree = BallTree(coords, metric="haversine")

# 500 m radius in radians = 0.5 / EarthRadius
inds = tree.query_radius(coords, r=0.5/6371)

neighbor_map = {int(boro_sites.iloc[i]["Site Number"]): 
                [int(boro_sites.iloc[j]["Site Number"]) for j in idxs if j!=i]
                for i, idxs in enumerate(inds)}
pd.Series(neighbor_map).to_json("data/neighbor_map_500m.json")
